In [1]:
# url = ""

In [4]:
import requests
from bs4 import BeautifulSoup
import json
from lxml import html
import time
import pandas as pd

In [31]:
NOT_FOUND = 404
states = ["U0" + str(u) for u in range(1, 8)] + ["S0" + str(s) if s < 10 else "S" + str(s) for s in range(1, 30)]

# base_url = "http://results.eci.gov.in/pc/en/constituencywise/ConstituencywiseS2910.htm?ac=10"

base_url = "http://results.eci.gov.in/PcResultGenJune2024/Constituencywise"

results = list()

In [6]:
response = requests.get("https://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseS071.htm")

In [7]:
url = base_url + states[0] + str(1) + ".htm?ac=" + str(1)

In [8]:
response = requests.get(url)

In [9]:
response_text = response.text

soup = BeautifulSoup(response_text, 'lxml')

In [10]:
tbodies = list(soup.find_all("tbody"))

In [11]:
tbody = tbodies[0]

In [1]:
#tbody

In [68]:
url

'http://results.eci.gov.in/PcResultGenJune2024/ConstituencywiseU011.htm?ac=1'

In [14]:
response_text = response.text

soup = BeautifulSoup(response_text, 'lxml')

In [15]:
tbodies = list(soup.find_all("h2"))


In [16]:
tbodies

[<h2>Parliamentary  Constituency <span> 1 - Andaman &amp; Nicobar Islands<strong> (Andaman &amp; Nicobar Islands)</strong></span> </h2>]

In [33]:
xx = tbodies[0]

In [41]:
xx.find_all("span")[0].text.strip().lower()

'1 - ambala (haryana)'

In [23]:
tbodies = list(soup.find_all("tfoot"))

In [30]:
tbodies[0].find_all("tr")[0].find_all("th")[3].text.strip().lower()

'201992'

In [17]:
tbodies[0].find_all("tr")[0].find_all("th")[3].text.strip().lower()

IndexError: list index out of range

In [15]:
tbody = tbodies[0]

In [16]:
trs = list(tbody.find_all('tr'))

In [17]:
trs

[<tr><td>1</td><td align="left">VARUN CHAUDHRY</td><td align="left">Indian National Congress</td><td align="right">662592</td><td align="right">1065</td><td align="right">663657</td><td align="right">49.28</td></tr>,
 <tr><td>2</td><td align="left">BANTO KATARIA</td><td align="left">Bharatiya Janata Party</td><td align="right">613786</td><td align="right">835</td><td align="right">614621</td><td align="right">45.64</td></tr>,
 <tr><td>3</td><td align="left">PAWAN RANDHAWA</td><td align="left">Bahujan Samaj Party</td><td align="right">23535</td><td align="right">48</td><td align="right">23583</td><td align="right">1.75</td></tr>,
 <tr><td>4</td><td align="left">GURPREET SINGH</td><td align="left">Indian National Lok Dal</td><td align="right">8902</td><td align="right">84</td><td align="right">8986</td><td align="right">0.67</td></tr>,
 <tr><td>5</td><td align="left">KAMAL KUMAR BARARA</td><td align="left">Aazad Samaj Party (Kanshi Ram)</td><td align="right">8741</td><td align="right">13

In [18]:
ind = 0 

In [24]:
tr = trs[1]

In [25]:
tr

<tr><td>2</td><td align="left">BANTO KATARIA</td><td align="left">Bharatiya Janata Party</td><td align="right">613786</td><td align="right">835</td><td align="right">614621</td><td align="right">45.64</td></tr>

In [22]:
tds = list(tr.find_all('td'))

In [26]:
tds[2].text.strip().lower()

'indian national congress'

In [58]:
df = pd.DataFrame({"Constituency","Code","candidate_name","party_name","evm_votes","post_votes","total_votes","vote_share"})



NameError: name 'pd' is not defined

In [ ]:
for state in states:
    
    for constituency_code in range(1, 99):
        url = base_url + state + str(constituency_code) + ".htm?ac=" + str(constituency_code)
        # print("URL", url)
        response = requests.get(url)

        # if some state and constituency combination do not exists, 404, continue for next state
        if NOT_FOUND == response.status_code:
            break

        response_text = response.text

        soup = BeautifulSoup(response_text, 'lxml')
        tbodies = list(soup.find_all("tbody"))
        # 11the tbody from top is the table we need to parse
        tbody = tbodies[10]
        trs = list(tbody.find_all('tr'))

        # all data for a constituency seat is stored in this dictionary
        seat = dict()
        seat["candidates"] = list()
        for tr_index, tr in enumerate(trs):

            # for rest of the rows, get data
            tds = list(tr.find_all('td'))

            candidate = dict()

            candidate["candidate_name"] = tds[1].text.strip().lower()
            candidate["party_name"] = tds[2].text.strip().lower()
            candidate["evm_votes"] = int(tds[3].text.strip().lower())
  
            candidate["post_votes"] = int(tds[4].text.strip().lower())
            candidate["total_votes"] = int(tds[5].text.strip().lower())
            candidate["share"] = float(tds[6].text.strip().lower())
                

        seat["candidates"].append(candidate)

    # print(json.dumps(seat, indent=2))
    results.append(seat)
    print("Collected data for", seat["state"], state, seat["constituency"], constituency_code, len(results))
    # Do not send too many hits to server. be gentle. wait.
    time.sleep(0.5)

# write data to file
with open("election_data.json", "a+") as f:
    f.write(json.dumps(results, indent=2))

In [64]:
df = pd.DataFrame(columns=["state","constituency_code","candidate_name","party_name","evm_votes","post_votes","total_votes","vote_share"])


In [65]:
df

,state,constituency_code,candidate_name,party_name,evm_votes,post_votes,total_votes,vote_share


In [66]:


for state in states:
    
    for constituency_code in range(1, 99):
        
        url = base_url + state + str(constituency_code) + ".htm?ac=" + str(constituency_code)

        response = requests.get(url)

        # if some state and constituency combination do not exists, 404, continue for next state
        if NOT_FOUND == response.status_code:
            break

        response_text = response.text

        soup = BeautifulSoup(response_text, 'lxml')
        
        title = list(soup.find_all("h2"))
        constituency_title = title[0].find_all("span")[0].text.strip().lower()
        
        tbodies = list(soup.find_all("tbody"))
        # 11the tbody from top is the table we need to parse
        tbody = tbodies[0]
        trs = list(tbody.find_all('tr'))

        # all data for a constituency seat is stored in this dictionary

        for tr_index, tr in enumerate(trs):

            # for rest of the rows, get data
            tds = list(tr.find_all('td'))
            if(tds[3].text.strip().lower()=="-"):
                evm_votes = 0 
            else:
                evm_votes = int(tds[3].text.strip().lower())
                
            if(tds[4].text.strip().lower()=="-"):
                post_votes = 0 
            else:
                post_votes = int(tds[4].text.strip().lower())        
           
            if(tds[5].text.strip().lower()=="-"):
                total_votes = 0 
            else:
                total_votes = int(tds[5].text.strip().lower())
                
            if(tds[6].text.strip().lower()=="-"):
                vote_share = 0 
            else:
                vote_share = float(tds[6].text.strip().lower())        
            
            df = pd.concat([df,pd.DataFrame({
                "state":state,
                "constituency_code":constituency_code,
                "constituency":constituency_title,
                "candidate_name":tds[1].text.strip().lower(),
                "party_name":tds[2].text.strip().lower(),
                "evm_votes" :evm_votes,
                "post_votes":post_votes,
                "total_votes":total_votes,
                "vote_share":vote_share
                
                },index=[0])],axis=0)
         
        ## total votes in a constituency    
        tfoot = list(soup.find_all("tfoot"))
        
        if(tfoot[0].find_all("tr")[0].find_all("th")[3].text.strip().lower()=="-"):
            evm_votes = 0 
        else:
            evm_votes = int(tfoot[0].find_all("tr")[0].find_all("th")[3].text.strip().lower())
                
        if(tfoot[0].find_all("tr")[0].find_all("th")[4].text.strip().lower()=="-"):
                post_votes = 0 
        else:
                post_votes = int(tfoot[0].find_all("tr")[0].find_all("th")[4].text.strip().lower())        
           
        if(tfoot[0].find_all("tr")[0].find_all("th")[5].text.strip().lower()=="-"):
            total_votes = 0 
        else:
            total_votes = int(tfoot[0].find_all("tr")[0].find_all("th")[5].text.strip().lower())
        
        
        df = pd.concat([df,pd.DataFrame({
             "state":state,
                "constituency_code":constituency_code,
                "constituency":constituency_title,
                "candidate_name":"Total",
                "party_name":"Overall",
                "evm_votes" :evm_votes,
                "post_votes":post_votes,
                "total_votes":total_votes,
                "vote_share":100.0
                
                },index=[0])],axis=0)
            
    print("Collected data for", constituency_title)
    # Do not send too many hits to server. be gentle. wait.
    time.sleep(0.5)
    

Collected data for 1 - andaman & nicobar islands (andaman & nicobar islands)
Collected data for 1 - chandigarh (chandigarh)
Collected data for 2 - dadar & nagar haveli (dadra & nagar haveli and daman & diu)
Collected data for 2 - dadar & nagar haveli (dadra & nagar haveli and daman & diu)
Collected data for 7 - south delhi (nct of delhi)
Collected data for 1 - lakshadweep (lakshadweep)
Collected data for 1 - puducherry (puducherry)
Collected data for 25 - chittoor(sc) (andhra pradesh)
Collected data for 2 - arunachal east (arunachal pradesh)
Collected data for 14 - jorhat (assam)
Collected data for 40 - jamui (bihar)
Collected data for 2 - south goa (goa)
Collected data for 26 - valsad (gujarat)
Collected data for 10 - faridabad (haryana)
Collected data for 4 - shimla (himachal pradesh)
Collected data for 4 - shimla (himachal pradesh)
Collected data for 28 - kolar (karnataka)
Collected data for 20 - thiruvananthapuram (kerala)
Collected data for 29 - betul (madhya pradesh)
Collected da

In [67]:
df.to_csv("/media/shivang/5A8AEAE28AEABA21/Users/shivang/Documents/election_2024.csv",index=False)

In [68]:
df['constituency'].nunique()

537